In [ ]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex AI 模型花园 TFVision 与图像分割

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/model_garden/model_garden_tfvision_image_segmentation.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在 Colab 中运行
    </a>
  </td>

  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/model_garden/model_garden_tfvision_image_segmentation.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在 GitHub 上查看
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/notebooks/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/community/model_garden/model_garden_tfvision_image_segmentation.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
在 Vertex AI Workbench 中打开
    </a>
  </td>
</table>

注意：此笔记本已在以下环境中进行测试：

* Python版本 = 3.9

## 概述

这本笔记本演示了如何在Vertex AI模型花园中使用[TFVision](https://github.com/tensorflow/models/blob/master/official/vision/MODEL_GARDEN.md)。

### 目标

* 训练新模型
  * 将输入数据转换为训练格式
  * 创建[超参数调整作业](https://cloud.google.com/vertex-ai/docs/training/hyperparameter-tuning-overview)来训练新模型
  * 查找并导出最佳模型

* 测试训练好的模型
  * 上传模型到模型注册表
  * 部署已上传的模型
  * 运行预测

* 清理资源

### 成本

本教程使用谷歌云的收费组件：

* Vertex AI
* 云存储

了解[Vertex AI 价格](https://cloud.google.com/vertex-ai/pricing)和[云存储 价格](https://cloud.google.com/storage/pricing)，并使用[价格计算器](https://cloud.google.com/products/calculator/)根据您的预期使用量生成费用估算。

在你开始之前

只限Colab

如果你使用工作台，请跳过本节并运行以下命令。

In [ ]:
if "google.colab" in str(get_ipython()):
    ! pip3 install --upgrade google-cloud-aiplatform

    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

    from google.colab import auth as google_auth

    google_auth.authenticate_user()

### 设置您的谷歌云项目

**无论您使用哪种笔记本环境，都需要按照以下步骤进行。**

1. [选择或创建一个谷歌云项目](https://console.cloud.google.com/cloud-resource-manager)。当您第一次创建帐户时，您将获得$300的免费信用以抵消计算/存储成本。

1. [确保为您的项目启用计费](https://cloud.google.com/billing/docs/how-to/modify-project)。

1. [启用Vertex AI API和Compute Engine API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com,compute_component)。

1. 如果您在本地运行此笔记本，您将需要安装[Cloud SDK](https://cloud.google.com/sdk)。

1. 在下面的单元格中输入您的项目ID。然后运行单元格，以确保Cloud SDK在此笔记本中的所有命令中使用正确的项目。

**注意**：Jupyter将以`!`为前缀的行视为shell命令，并将以`$`为前缀的Python变量插入这些命令中。

In [ ]:
import os

from google.cloud import aiplatform

# The project and bucket are for experiments below.
PROJECT_ID = ""  # @param {type:"string"}
BUCKET_URI = ""  # @param {type:"string"}

# You can choose a region from https://cloud.google.com/about/locations.
# Only regions prefixed by "us", "asia", or "europe" are supported.
REGION = "europe-west4"  # @param {type:"string"}
REGION_PREFIX = REGION.split("-")[0]
assert REGION_PREFIX in (
    "us",
    "europe",
    "asia",
), f'{REGION} is not supported. It must be prefixed by "us", "asia", or "europe".'

! gcloud config set project $PROJECT_ID

STAGING_BUCKET = os.path.join(BUCKET_URI, "temporal")
CHECKPOINT_BUCKET = os.path.join(BUCKET_URI, "ckpt")

aiplatform.init(project=PROJECT_ID, location=REGION, staging_bucket=STAGING_BUCKET)

# Download config files.
CONFIG_DIR = os.path.join(BUCKET_URI, "config")
! wget https://raw.githubusercontent.com/tensorflow/models/master/official/vision/configs/experiments/semantic_segmentation/deeplabv3plus_resnet101_cityscapes_gpu_multiworker_mirrored.yaml
! gsutil cp deeplabv3plus_resnet101_cityscapes_gpu_multiworker_mirrored.yaml $CONFIG_DIR/

###定义常量

In [ ]:
OBJECTIVE = "isg"

# Data converter constants.
DATA_CONVERTER_JOB_PREFIX = "data_converter"
DATA_CONVERTER_CONTAINER = f"{REGION_PREFIX}-docker.pkg.dev/vertex-ai/vertex-vision-model-garden-dockers/data-converter"
DATA_CONVERTER_MACHINE_TYPE = "n1-highmem-8"


# Training constants.
TRAINING_JOB_PREFIX = "train"
TRAIN_CONTAINER_URI = f"{REGION_PREFIX}-docker.pkg.dev/vertex-ai/vertex-vision-model-garden-dockers/tfvision-oss"
TRAIN_MACHINE_TYPE = "n1-highmem-16"
TRAIN_ACCELERATOR_TYPE = "NVIDIA_TESLA_V100"
TRAIN_NUM_GPU = 2
TRAIN_DEEPLABV3PLUS_CONFIG = os.path.join(
    CONFIG_DIR, "deeplabv3plus_resnet101_cityscapes_gpu_multiworker_mirrored.yaml"
)

# Evaluation constants.
EVALUATION_METRIC = "mean_iou"

# Export constants.
EXPORT_JOB_PREFIX = "export"
EXPORT_CONTAINER_URI = f"{REGION_PREFIX}-docker.pkg.dev/vertex-ai-restricted/vertex-vision-model-garden-dockers/tfvision-model-export"
EXPORT_MACHINE_TYPE = "n1-highmem-8"

# Prediction constants.
# You can deploy models with
#   pre-build-dockers: https://cloud.google.com/vertex-ai/docs/predictions/pre-built-containers.
#   and optimized tensorflow runtime dockers: https://cloud.google.com/vertex-ai/docs/predictions/optimized-tensorflow-runtime.
# The example in this notebook uses optimized tensorflow runtime dockers.
# You can adjust accelerator types and machine types to get faster predictions.
PREDICTION_CONTAINER_URI = f"{REGION_PREFIX}-docker.pkg.dev/vertex-ai-restricted/prediction/tf_opt-gpu.2-11:latest"
SERVING_CONTAINER_ARGS = ["--allow_precompilation", "--allow_compression"]
PREDICTION_ACCELERATOR_TYPE = "NVIDIA_TESLA_T4"
PREDICTION_MACHINE_TYPE = "n1-standard-4"
UPLOAD_JOB_PREFIX = "upload"
DEPLOY_JOB_PREFIX = "deploy"

### 定义常用库

In [ ]:
import base64
import json
import os
from datetime import datetime
from io import BytesIO
from typing import Dict, List, Union

import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import yaml
from google.cloud import aiplatform
from google.protobuf import json_format
from google.protobuf.struct_pb2 import Value
from PIL import Image


def get_job_name_with_datetime(prefix: str):
    return prefix + datetime.now().strftime("_%Y%m%d_%H%M%S")


def predict_custom_trained_model(
    project: str,
    endpoint_id: str,
    instances: Union[Dict, List[Dict]],
    location: str = "us-central1",
):
    # The AI Platform services require regional API endpoints.
    client_options = {"api_endpoint": f"{location}-aiplatform.googleapis.com"}
    # Initialize client that will be used to create and send requests.
    # This client only needs to be created once, and can be reused for multiple requests.
    client = aiplatform.gapic.PredictionServiceClient(client_options=client_options)
    parameters_dict = {}
    parameters = json_format.ParseDict(parameters_dict, Value())
    endpoint = client.endpoint_path(
        project=project, location=location, endpoint=endpoint_id
    )
    response = client.predict(
        endpoint=endpoint, instances=instances, parameters=parameters
    )
    return response.predictions, response.deployed_model_id


def load_img(path):
    img = tf.io.read_file(path)
    img = tf.image.decode_jpeg(img, channels=3)
    return Image.fromarray(np.uint8(img)).convert("RGB")


def display_image(original_image, category_image_color, score_image_grayscale):
    _, axarr = plt.subplots(1, 3, figsize=(20, 15))
    axarr[0].imshow(original_image)
    axarr[1].imshow(category_image_color)
    axarr[2].imshow(score_image_grayscale.convert("RGB"))


def get_prediction_instances(test_filepath, new_width=-1):
    if new_width <= 0:
        with tf.io.gfile.GFile(test_filepath, "rb") as input_file:
            encoded_string = base64.b64encode(input_file.read()).decode("utf-8")
    else:
        img = load_img(test_filepath)
        width, height = img.size
        print("original input image size: ", width, " , ", height)
        new_height = int(height * new_width / width)
        new_img = img.resize((new_width, new_height))
        print("resized input image size: ", new_width, " , ", new_height)
        buffered = BytesIO()
        new_img.save(buffered, format="JPEG")
        encoded_string = base64.b64encode(buffered.getvalue()).decode("utf-8")

    instances = [
        {
            "encoded_image": {"b64": encoded_string},
        }
    ]
    return instances


def get_label_map(label_map_yaml_filepath):
    with tf.io.gfile.GFile(label_map_yaml_filepath, "rb") as input_file:
        label_map = yaml.safe_load(input_file.read())
    return label_map


def get_best_trial(model_dir, max_trial_count, evaluation_metric):
    best_trial_dir = ""
    best_trial_evaluation_results = {}
    best_performance = -1

    for i in range(max_trial_count):
        current_trial = i + 1
        current_trial_dir = os.path.join(model_dir, "trial_" + str(current_trial))
        current_trial_best_ckpt_dir = os.path.join(current_trial_dir, "best_ckpt")
        current_trial_best_ckpt_evaluation_filepath = os.path.join(
            current_trial_best_ckpt_dir, "info.json"
        )
        with tf.io.gfile.GFile(current_trial_best_ckpt_evaluation_filepath, "rb") as f:
            eval_metric_results = json.load(f)
            current_performance = eval_metric_results[evaluation_metric]
            if current_performance > best_performance:
                best_performance = current_performance
                best_trial_dir = current_trial_dir
                best_trial_evaluation_results = eval_metric_results
    return best_trial_dir, best_trial_evaluation_results


def create_coco_stuff_label_colormap():
    """Creates a label colormap used in COCO-Stuff segmentation benchmark.

    Returns:
      A colormap for visualizing segmentation results.
    """
    return np.asarray(
        [
            [54, 178, 118],
            [0, 85, 178],
            [150, 178, 22],
            [107, 0, 0],
            [0, 0, 89],
            [0, 117, 178],
            [47, 178, 124],
            [178, 116, 0],
            [0, 0, 178],
            [79, 178, 92],
            [134, 0, 0],
            [22, 178, 150],
            [178, 87, 0],
            [178, 146, 0],
            [0, 5, 178],
            [0, 0, 125],
            [0, 53, 178],
            [0, 132, 178],
            [111, 178, 60],
            [178, 131, 0],
            [0, 29, 178],
            [178, 109, 0],
            [178, 35, 0],
            [0, 148, 178],
            [9, 172, 163],
            [0, 0, 178],
            [178, 124, 0],
            [178, 102, 0],
            [0, 156, 175],
            [178, 43, 0],
            [0, 0, 170],
            [178, 94, 0],
            [0, 0, 134],
            [67, 178, 105],
            [99, 178, 73],
            [0, 37, 178],
            [86, 178, 86],
            [15, 178, 156],
            [0, 0, 152],
            [178, 21, 0],
            [0, 124, 178],
            [0, 61, 178],
            [178, 50, 0],
            [0, 109, 178],
            [137, 178, 35],
            [0, 13, 178],
            [0, 101, 178],
            [0, 0, 116],
            [0, 45, 178],
            [41, 178, 131],
            [0, 0, 161],
            [178, 72, 0],
            [0, 0, 143],
            [116, 0, 0],
            [28, 178, 143],
            [170, 6, 0],
            [156, 178, 15],
            [89, 0, 0],
            [143, 178, 28],
            [73, 178, 99],
            [118, 178, 54],
            [92, 178, 79],
            [152, 0, 0],
            [178, 153, 0],
            [98, 0, 0],
            [178, 65, 0],
            [60, 178, 111],
            [169, 175, 3],
            [105, 178, 67],
            [178, 13, 0],
            [163, 178, 9],
            [3, 164, 169],
            [125, 0, 0],
            [175, 168, 0],
            [178, 138, 0],
            [178, 28, 0],
            [35, 178, 137],
            [0, 140, 178],
            [0, 0, 98],
            [131, 178, 41],
            [0, 77, 178],
            [0, 0, 107],
            [0, 93, 178],
            [143, 0, 0],
            [178, 58, 0],
            [161, 0, 0],
            [0, 69, 178],
            [178, 160, 0],
            [178, 80, 0],
            [0, 21, 178],
            [124, 178, 47],
            [255, 214, 0],
        ]
    )


def parse_segmentation_prediction(prediction):
    score_bytes = prediction["score_bytes"]
    score_image_grayscale = Image.open(
        BytesIO(base64.b64decode(dict(score_bytes)["b64"]))
    )
    category_bytes = prediction["category_bytes"]
    category_image_grayscale = Image.open(
        BytesIO(base64.b64decode(dict(category_bytes)["b64"]))
    )

    # Visualize category images.
    color_map = create_coco_stuff_label_colormap()
    category_image_grayscale_np = np.array(category_image_grayscale)
    rendered_image_shape = category_image_grayscale_np.shape + (3,)
    category_image_color_np = np.zeros(rendered_image_shape, dtype=np.uint8)
    unique_labels = np.unique(category_image_grayscale_np)
    for label in unique_labels:
        if label == 0:
            continue
        category_image_color_np[category_image_grayscale_np == label] = color_map[
            label % len(color_map)
        ]
    category_image_color = Image.fromarray(category_image_color_np)

    return score_image_grayscale, category_image_color


def upload_checkpoint_to_gcs(checkpoint_url):
    filename = os.path.basename(checkpoint_url)
    checkpoint_name = filename.replace(".tar.gz", "")
    print("Download checkpoint from", checkpoint_url, "and store to", CHECKPOINT_BUCKET)
    ! wget $checkpoint_url -O $filename
    ! mkdir -p $checkpoint_name
    ! tar -xvzf $filename -C $checkpoint_name

    # Search for relative path to the checkpoint.
    checkpoint_path = None
    for root, dirs, files in os.walk(checkpoint_name):
        for file in files:
            if file.endswith(".index"):
                checkpoint_path = os.path.join(root, os.path.splitext(file)[0])
                checkpoint_path = os.path.relpath(checkpoint_path, checkpoint_name)
                break

    ! gsutil cp -r $checkpoint_name $CHECKPOINT_BUCKET/
    checkpoint_uri = os.path.join(CHECKPOINT_BUCKET, checkpoint_name, checkpoint_path)
    print("Checkpoint uploaded to", checkpoint_uri)
    return checkpoint_uri

## 训练新模型
本节展示如何训练新模型。
1. 将输入数据转换为训练格式
2. 创建超参数调整作业来训练新模型
3. 查找并导出最佳模型

如果您已经训练过模型，请转至“测试训练过的模型”部分。

### 为训练准备输入数据

按照[这里](https://cloud.google.com/vertex-ai/docs/image-data/classification/prepare-data)描述的格式准备数据，然后将它们转换为以下的训练格式：

* `input_file_path`：coco json格式的输入文件路径。
* `split_ratio`：要拆分为训练/验证/测试数据的比例。
* `num_shard`：用于训练/验证/测试的碎片数量。
* `output_dir`：输出目录，其中包含准备好的训练/测试/验证数据。

In [ ]:
# This job will convert input data as training format, with given split ratios
# and number of shards on train/test/validation.
data_converter_job_name = get_job_name_with_datetime(
    DATA_CONVERTER_JOB_PREFIX + "_" + OBJECTIVE
)

input_file_path = ""  # @param {type:"string"}
split_ratio = "0.8,0.1,0.1"
num_shard = "10,10,10"
data_converter_output_dir = os.path.join(BUCKET_URI, data_converter_job_name)


worker_pool_specs = [
    {
        "machine_spec": {
            "machine_type": DATA_CONVERTER_MACHINE_TYPE,
        },
        "replica_count": 1,
        "container_spec": {
            "image_uri": DATA_CONVERTER_CONTAINER,
            "command": [],
            "args": [
                "--input_file_path=%s" % input_file_path,
                "--input_file_type=coco_json",
                "--objective=%s" % OBJECTIVE,
                "--num_shard=%s" % num_shard,
                "--split_ratio=%s" % split_ratio,
                "--output_dir=%s" % data_converter_output_dir,
            ],
        },
    }
]

data_converter_custom_job = aiplatform.CustomJob(
    display_name=data_converter_job_name,
    project=PROJECT_ID,
    worker_pool_specs=worker_pool_specs,
    staging_bucket=STAGING_BUCKET,
)

data_converter_custom_job.run()

input_train_data_path = os.path.join(data_converter_output_dir, "train.tfrecord*")
input_validation_data_path = os.path.join(data_converter_output_dir, "val.tfrecord*")
label_map_path = os.path.join(data_converter_output_dir, "label_map.yaml")
print("input_train_data_path for training: ", input_train_data_path)
print("input_validation_data_path for training: ", input_validation_data_path)
print("label_map_path for prediction: ", label_map_path)

### 使用超参数调整创建 Vertex AI 定制作业

您可以使用 Vertex AI SDK 创建并运行具有 Vertex AI Model Garden Training Dockers 的超参数调整作业。

#### 定义以下规范
* `worker_pool_specs`：指定机器类型和Docker镜像的字典。此示例定义了一个包含一个`n1-standard-4`机器和两个`NVIDIA_TESLA_T4` GPU的单节点集群。
* `parameter_spec`：指定要优化的参数的字典。字典键是分配给训练应用程序代码中每个超参数的命令行参数的字符串，字典值是参数规范。参数规范包括超参数的类型、最小/最大值和缩放。
* `metric_spec`：指定要优化的度量标准的字典。字典键是您在训练应用程序代码中设置的`hyperparameter_metric_tag`，值是优化目标。

In [ ]:
from google.cloud.aiplatform import hyperparameter_tuning as hpt

label_map = get_label_map(label_map_path)
num_classes = len(label_map["label_map"]) + 1

# Input train and validation datasets can be found from the section above
# `Convert input data for training`.
# Set prepared datasets if exists.
# input_train_data_path = ''
# input_validation_data_path = ''

# Refer to https://github.com/tensorflow/models/blob/master/official/vision/MODEL_GARDEN.md
# for more model details.
experiment = "deeplabv3plus"  # @param ["deeplabv3plus"]

train_job_name = get_job_name_with_datetime(TRAINING_JOB_PREFIX + "_" + OBJECTIVE)
model_dir = os.path.join(BUCKET_URI, train_job_name)

# The arguments here are mainly for test purposes. Please update them
# to get better performances.
experiment_container_args_dict = {
    # deeplabv3plus experiment args.
    "deeplabv3plus": {
        "experiment": "seg_deeplabv3plus_pascal",
        "config_file": TRAIN_DEEPLABV3PLUS_CONFIG,
        "input_train_data_path": input_train_data_path,
        "input_validation_data_path": input_validation_data_path,
        "objective": OBJECTIVE,
        "model_dir": model_dir,
        "num_classes": num_classes,
        "global_batch_size": 2,
        "prefetch_buffer_size": 12,
        "train_steps": 500,
        "output_size": "1024,2048",
        "init_checkpoint": "https://storage.googleapis.com/tf_model_garden/vision/deeplabv3plus/dilated-resnet-101-deeplabv3plus.tar.gz",
    }
}
experiment_container_args = experiment_container_args_dict[experiment]

# Copy checkpoint to GCS bucket if specified.
init_checkpoint = experiment_container_args.get("init_checkpoint")
if init_checkpoint:
    experiment_container_args["init_checkpoint"] = upload_checkpoint_to_gcs(
        init_checkpoint
    )

worker_pool_specs = [
    {
        "machine_spec": {
            "machine_type": TRAIN_MACHINE_TYPE,
            "accelerator_type": TRAIN_ACCELERATOR_TYPE,
            "accelerator_count": TRAIN_NUM_GPU,
        },
        "replica_count": 1,
        "container_spec": {
            "image_uri": TRAIN_CONTAINER_URI,
            "args": [
                "--mode=train_and_eval",
            ]
            + ["--{}={}".format(k, v) for k, v in experiment_container_args.items()],
        },
    },
]

metric_spec = {"model_performance": "maximize"}

LEARNING_RATES = [0.001]
# Models will be trained with each learning rate separately and max trial count is the number of learning rates.
MAX_TRIAL_COUNT = len(LEARNING_RATES)
parameter_spec = {
    "learning_rate": hpt.DiscreteParameterSpec(values=LEARNING_RATES, scale="linear"),
}

print(worker_pool_specs, metric_spec, parameter_spec)

#### 运行超参数调整作业
* `max_trial_count`: 设置服务将运行的试验数量的上限。推荐的做法是从较少数量的试验开始，了解您选择的超参数对结果的影响程度，然后再逐渐增加。

* `parallel_trial_count`: 如果使用并行试验，服务将为多个训练处理集群提供资源。在创建作业时指定的工作池规范将用于每个单独的训练集群。增加并行试验的数量可缩短超参数调整作业运行所需的时间；但是，这可能会降低作业的整体有效性。因为默认的调整策略使用先前试验的结果来指导后续试验中数值的指定。

* `search_algorithm`: 可用的搜索算法有grid、random或默认值（None）。默认选项应用贝叶斯优化来搜索可能超参数值的空间，是推荐的算法。

单击输出中生成的链接以在Cloud控制台中查看您的运行。

In [ ]:
train_custom_job = aiplatform.CustomJob(
    display_name=train_job_name,
    project=PROJECT_ID,
    worker_pool_specs=worker_pool_specs,
    staging_bucket=STAGING_BUCKET,
)

train_hpt_job = aiplatform.HyperparameterTuningJob(
    display_name=train_job_name,
    custom_job=train_custom_job,
    metric_spec=metric_spec,
    parameter_spec=parameter_spec,
    max_trial_count=MAX_TRIAL_COUNT,
    parallel_trial_count=1,
    project=PROJECT_ID,
    search_algorithm=None,
)

train_hpt_job.run()

print("experiment is: ", experiment)
print("model_dir is: ", model_dir)

将最佳模型导出为TF Saved Model格式

In [ ]:
# This job will export models from TF checkpoints to TF saved model format.
from google.cloud import aiplatform

# model_dir is from the section above.
best_trial_dir, best_trial_evaluation_results = get_best_trial(
    model_dir, MAX_TRIAL_COUNT, EVALUATION_METRIC
)
print("best_trial_dir: ", best_trial_dir)
print("best_trial_evaluation_results: ", best_trial_evaluation_results)

worker_pool_specs = [
    {
        "machine_spec": {
            "machine_type": EXPORT_MACHINE_TYPE,
        },
        "replica_count": 1,
        "container_spec": {
            "image_uri": EXPORT_CONTAINER_URI,
            "command": [],
            "args": [
                "--objective=%s" % OBJECTIVE,
                "--experiment=%s" % experiment_container_args["experiment"],
                "--config_file=%s/params.yaml" % best_trial_dir,
                "--checkpoint_path=%s/best_ckpt" % best_trial_dir,
                "--export_dir=%s/best_model" % model_dir,
                "--input_image_size=%s" % experiment_container_args["output_size"],
            ],
        },
    }
]

model_export_name = get_job_name_with_datetime(EXPORT_JOB_PREFIX + "_" + OBJECTIVE)
model_export_custom_job = aiplatform.CustomJob(
    display_name=model_export_name,
    project=PROJECT_ID,
    worker_pool_specs=worker_pool_specs,
    staging_bucket=STAGING_BUCKET,
)


model_export_custom_job.run()

print("best model is saved to: ", os.path.join(model_dir, "best_model"))

## 测试训练好的模型
这部分展示了如何使用训练好的模型进行测试。
1. 上传和部署模型
2. 运行预测

In [ ]:
# @title Upload and deploy models
# model_dir is from the section above.
trained_model_dir = os.path.join(model_dir, "best_model/saved_model")

upload_job_name = get_job_name_with_datetime(UPLOAD_JOB_PREFIX + "_" + OBJECTIVE)

serving_env = {
    "MODEL_ID": "deeplabv3plus-cityscapes-20230315",
    "DEPLOY_SOURCE": "notebook",
}

model = aiplatform.Model.upload(
    display_name=upload_job_name,
    artifact_uri=trained_model_dir,
    serving_container_image_uri=PREDICTION_CONTAINER_URI,
    serving_container_args=SERVING_CONTAINER_ARGS,
    serving_container_environment_variables=serving_env,
)

model.wait()

print("The uploaded model name is: ", upload_job_name)

deploy_model_name = get_job_name_with_datetime(DEPLOY_JOB_PREFIX + "_" + OBJECTIVE)
print("The deployed job name is: ", deploy_model_name)

endpoint = model.deploy(
    deployed_model_display_name=deploy_model_name,
    machine_type=PREDICTION_MACHINE_TYPE,
    traffic_split={"0": 100},
    accelerator_type=PREDICTION_ACCELERATOR_TYPE,
    accelerator_count=1,
    min_replica_count=1,
    max_replica_count=1,
)

endpoint_id = endpoint.name
print("endpoint id is: ", endpoint_id)

In [ ]:
# @title Run predictions
# endpoint_id was generated in the section above (`Upload and deploy models`).
endpoint_id = endpoint.name

# The test image file path.
test_filepath = ""  # @param {type:"string"}
score_threshold = 0.5  # @param {type:"number"}
# If the input image is too large, we will resize it for prediction.
instances = get_prediction_instances(test_filepath, new_width=1000)

# The label map file was generated from the section above (`Convert input data for training`).
label_map = get_label_map(label_map_path)["label_map"]

predictions, _ = predict_custom_trained_model(
    project=PROJECT_ID, location=REGION, endpoint_id=endpoint_id, instances=instances
)

score_image_grayscale, category_image_color = parse_segmentation_prediction(
    dict(predictions[0])
)
display_image(
    load_img(test_filepath), category_image_color, score_image_grayscale.convert("RGB")
)

清理

In [ ]:
# Undeploy model and delete endpoint.
endpoint.delete(force=True)
# Delete models.
model.delete()
# Delete custom and hpt jobs.
if data_converter_custom_job.list(filter=f'display_name="{data_converter_job_name}"'):
    data_converter_custom_job.delete()
if train_hpt_job.list(filter=f'display_name="{train_job_name}"'):
    train_hpt_job.delete()
if model_export_custom_job.list(filter=f'display_name="{model_export_name}"'):
    model_export_custom_job.delete()